# Morse Coefficient Optimization
## By Terry Bondy, VA3TYB

In [1]:
printf(strftime ("Last updated: %A %e %B %Y", localtime (time ())))

Last updated: Tuesday 10 December 2019

The notebook concerns itself with optimizing the detection coefficients $\phi_d(i)$ from `MorseCoefficients` notebook where $\phi_d(i)$ is is the $i^{th}$ detection optimizing phase value for the message.

Turns out if $M$ is the number of bits, then only have to optimize $\frac{M}{2}$ values, because for maximal detection, $\phi_d(i) = -\phi_d(M + 1 - i)$


The following function computes a column vector representing the sound/silence coefficients $a_s(i)$ for the message to be sent.

In [ ]:
function A_s = makeMorseSoundSilence(message)
    baseCoeff = alphaToMorse(message)';
    A_s = vertcat(baseCoeff, [ 0; 0; 0; 0; 0; 0; 0; ], flip(baseCoeff));
    sz = rows(A_s);
    # See if needs padding
    if (sz < 551)
        half = (551 - sz)/2;
        A_s = vertcat(zeros(half,1), baseCoeff, [ 0; 0; 0; 0; 0; 0; 0; ], flip(baseCoeff), zeros(half,1));
    endif        
endfunction

Try it out

In [ ]:
A_s = makeMorseSoundSilence("QRG DE VA3TYB VA3TYB?");
rows(A_s)

Determine the number of "on" bits.

In [ ]:
A_s' * A_s

The following is the cost function used to optimize $\phi_d(i)$.

In [ ]:
# Phi_d is a column vector, 0 <= phi <= pi, length m
# A_s is a column vector, 0 or 1s, length 2m + 1
function cost = costAny(Phi_d, A_s)
  # Make a test vector where the back half is a mirror and congugate of the front half for maximal detection
  # Centre value is 1, because it is the unit vector that is a congugate of itself
  test = A_s .* [ exp(j .* Phi_d); 1; exp(-j .* flip(Phi_d)) ];
  cost = max(abs(conv(test,flip(test))))/(A_s' * A_s);
endfunction

Try it out. Should get an answer between 0.15 and 0.5 but usually between 0.25 and 0.4.

In [ ]:
costAny(pi .* rand(floor(rows(A_s)/2), 1), A_s)

Ensure that you run `Nonlin_min_Install` notebook to install this package.

In [ ]:
pkg load optim

Finally a function that will provide $a_s(i)$ and $\phi_d(i)$ for a given message.

In [ ]:
function [A_s, Phi_d, objf, cvg] = optimize(message)
  A_s = makeMorseSoundSilence(message);
  m = floor(rows(A_s)/2);
  A_s_half = A_s(1:m);
  init_p = pi .* rand(m, 1) .* A_s_half;
  # Not varying all the parameters
  fixed = not(logical(A_s_half));
  lbound = zeros(m, 1);
  ubound = pi .* ones(m, 1);
  [Phi_d, objf, cvg] = nonlin_min (@ (Phi_trial) costAny(Phi_trial, A_s), init_p, optimset ("Algorithm", "samin", 
    "fixed", fixed,
    "lbound", lbound,
    "ubound", ubound,
    "Display", "iter"
  ));
endfunction

Try it out.

In [ ]:
[VA3ASE_A_s, VA3ASE_Phi_d, objf, cvg] = optimize("QRG DE VA3ASE VA3ASE?");
cvg
objf


In [ ]:
[VE3YRA_A_s, VE3YRA_Phi_d, objf, cvg] = optimize("QRG DE VE3YRA VE3YRA?");
cvg
objf

In [ ]:
[VA3TYB_A_s, VA3TYB_Phi_d, objf, cvg] = optimize("QRG DE VA3TYB VA3TYB?");
cvg
objf

In [ ]:
format long

In [ ]:
save "VA3ASE_A_s.mat" VA3ASE_A_s

In [ ]:
save "VA3ASE_Phi_d.mat" VA3ASE_Phi_d

In [ ]:
save "VE3YRA_A_s.mat" VE3YRA_A_s

In [ ]:
save "VE3YRA_Phi_d.mat" VE3YRA_Phi_d

In [ ]:
save "VA3TYB_A_s.mat" VE3YRA_A_s

In [ ]:
save "VA3TYB_Phi_d.mat" VA3TYB_Phi_d